# Library

In [242]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

# Data

### Historical data

In [76]:
df_hist_ESG = pd.read_csv('data/ESG Historic Data.csv', sep=';', encoding='latin-1')
df_hist_ESG.shape

(34672, 276)

In [3]:
df_hist_ESG.columns

Index(['Vigeo Key', 'ISIN', 'Bloomberg Legal entity identifier (LEI)',
       'Bloomberg Ticker', 'SEDOL code primary', 'SEDOL 2 code', 'Title',
       'Zone', 'Country', 'Generic Sector',
       ...
       'C&S score sector average', 'CIN score sector average',
       'CG score sector average', 'HRts score sector average',
       'HRS Domain rating', 'ENV Domain rating', 'C&S Domain rating',
       'CIN Domain rating', 'CGV Domain rating', 'HRT Domain rating'],
      dtype='object', length=276)

In [220]:
# ESG sub-criteria
global_score = ['ISIN', 'Global score']
global_score = ['ISIN', 'Global score']
ESG_criteria = ['Produced date','ISIN','ESG ENV score', 'ESG SOC score', 'ESG GOV score'] 
ESG_sub_criteria = ['Produced date','ISIN', 'HRS Domain score','ENV score','C&S score', 'CIN score', 'CG score', 'HRts score']
ESG_sub_sub_criteria = ['Produced date','ISIN', 
                        'HR L-score', 'HR I-score', 'HR R-score', 
                        'ENV L-score', 'ENV I-score','ENV R-score', 
                        'C&S L-score', 'C&S I-score', 'C&S R-score', 
                        'CIN score', 'CIN L-score', 'CIN I-score', 'CIN R-score',
                        'CG L-score', 'CG I-score', 'CG R-score', 
                        'HRts R-score', 'HRts I-score', 'HRts R-score']


### Financial data

In [77]:
df_financial_data = pd.read_excel('data/Financial DATA.xlsx')
df_financial_data.shape

(1541, 16)

In [78]:
usfull_col = ['Ticker', 'Pays/Région', 'Secteur']

In [79]:
df_financial_data.head(2)

,Titre,Ticker,Source,Position,Var pos,% circ,% Net,VM actu,Rpt MV,Var VM ac,Déposé le,Pays/Région,Secteur,Sous-secteur,Région,Type d'actif
0,Apple Inc,AAPL US,ETF,"1,259,255",28,0.01,4.53,181.50MLN,172.74MLN,"+4,035.79",03/15/22,Etats-Unis,Technologie,Matériel tech et semi-conducteurs,Am. du Nord,Actions
1,Microsoft Corp,MSFT US,ETF,547.454,13,0.01,3.62,142.16MLN,137.83MLN,"+3,375.77",03/15/22,Etats-Unis,Technologie,Logiciels et srvces tech,Am. du Nord,Actions


### Coresspondance table

In [135]:
df_corespondance = pd.read_excel('data/Correspondence Table.xlsx')
df_corespondance.shape

(4208, 24)

In [136]:
df_corespondance = df_corespondance[['ISSUER', 'ISIN','TICKER EQUITY', 'CDS 5 ANS']]

In [137]:
df_corespondance.isna().sum()

ISSUER           1691
ISIN             1691
TICKER EQUITY    2473
CDS 5 ANS        3033
dtype: int64

In [138]:
df_corespondance.dropna(subset=['TICKER EQUITY', 'ISSUER', 'CDS 5 ANS'], inplace = True)

In [83]:
df_corespondance.shape[0]

782

In [84]:
df_corespondance.head(3)

,ISSUER,ISIN,TICKER EQUITY,CDS 5 ANS
0,Apple Inc,US0378331005,AAPL US Equity,CY179834
1,Microsoft Corp,US5949181045,MSFT US Equity,CX399237
2,Amazon.com Inc,US0231351067,AMZN US Equity,CY372412


In [85]:
df_corespondance.isna().sum()

ISSUER           0
ISIN             0
TICKER EQUITY    0
CDS 5 ANS        0
dtype: int64

In [86]:
len(list(df_corespondance['TICKER EQUITY'].unique()))

781

In [87]:
def substract(a, b):                              
    return "".join(a.rsplit(b))

clean_ticker = []
for i in df_corespondance['TICKER EQUITY']:
    clean_ticker.append(substract(i, 'Equity'))
    
df_corespondance['ticker'] = clean_ticker


In [38]:
df_corespondance

,ISSUER,ISIN,TICKER EQUITY,CDS 5 ANS,ticker
0,Apple Inc,US0378331005,AAPL US Equity,CY179834,AAPL US
1,Microsoft Corp,US5949181045,MSFT US Equity,CX399237,MSFT US
2,Amazon.com Inc,US0231351067,AMZN US Equity,CY372412,AMZN US
3,Alphabet Inc,US02079K3059,GOOGL US Equity,CX410247,GOOGL US
4,Tesla Inc,US88160R1014,GOOG US Equity,CY371958,GOOG US
...,...,...,...,...,...
1728,SLOVAKIA GOVERNMENT BOND,SK4120009762,AFRY SS Equity,CSLVK1U5,AFRY SS
1730,CHINA GOVT INTL BOND,XS2078532913,ROO LN Equity,CCHIN1U5,ROO LN
1731,CIE FINANCEMENT FONCIER,FR0014002X50,IVG IM Equity,CY360464,IVG IM
1733,LANDWIRTSCH. RENTENBANK,XS1957349332,AG1 GR Equity,CY416840,AG1 GR


In [88]:
df_corespondance.rename(columns = {'ticker':'TICKER'}, inplace = True)

In [20]:
#df_corespondance.drop('TICKER', axis = 1, inplace = True)

In [89]:
df_corespondance.head(5)

,ISSUER,ISIN,TICKER EQUITY,CDS 5 ANS,TICKER
0,Apple Inc,US0378331005,AAPL US Equity,CY179834,AAPL US
1,Microsoft Corp,US5949181045,MSFT US Equity,CX399237,MSFT US
2,Amazon.com Inc,US0231351067,AMZN US Equity,CY372412,AMZN US
3,Alphabet Inc,US02079K3059,GOOGL US Equity,CX410247,GOOGL US
4,Tesla Inc,US88160R1014,GOOG US Equity,CY371958,GOOG US


### Equity data

In [116]:
df_equity = pd.read_csv('Data/MSCI WORLD TT RETURN DIV BRUT.csv', sep=';', encoding='latin-1')

In [117]:
df_equity.head(3)

,Dates,AAPL US Equity,MSFT US Equity,AMZN US Equity,GOOGL US Equity,GOOG US Equity,TSLA US Equity,NVDA US Equity,UNH US Equity,FB US Equity,...,3635 JP Equity,4506 JP Equity,MLCO US Equity,2412 JP Equity,7205 JP Equity,WEED CN Equity,FUTU US Equity,8439 JP Equity,4927 JP Equity,INPST NA Equity
0,02/01/2012,"-0,0296","-0,2306","-0,4371","0,5448",NaN,"-0,5917","-0,7874","-0,9769",NaN,...,"2,2581","-0,2275","2,7778","-0,9276","0,2146","-11,5385",NaN,"-0,6148","0,1444",NaN
1,03/01/2012,"1,5383","3,1009","3,4258","3,0206",NaN,"-1,6807","1,2987","1,5983",NaN,...,"2,2581","-0,2275","3,4304","-0,9276","0,2146","-11,5385",NaN,"-0,6148","0,1444",NaN
2,04/01/2012,"0,5374","2,3725","-0,849","0,4313",NaN,"-1,3177","1,1396","1,4566",NaN,...,"0,4732","0,5701","-1,3065",0,"-0,2141","-11,5385",NaN,"3,6426","0,0481",NaN


In [118]:
df_equity.shape

(2662, 1541)

In [119]:
df_equity_m=df_equity.set_index('Dates')

In [120]:
df_equity_m=df_equity_m.fillna(method='ffill')

In [121]:
df_equity_m=df_equity_m.fillna(0)
df_equity_m.head(5)

,AAPL US Equity,MSFT US Equity,AMZN US Equity,GOOGL US Equity,GOOG US Equity,TSLA US Equity,NVDA US Equity,UNH US Equity,FB US Equity,JNJ US Equity,...,3635 JP Equity,4506 JP Equity,MLCO US Equity,2412 JP Equity,7205 JP Equity,WEED CN Equity,FUTU US Equity,8439 JP Equity,4927 JP Equity,INPST NA Equity
Dates,,,,,,,,,,,,,,,,,,,,,
02/01/2012,"-0,0296","-0,2306","-0,4371","0,5448",0,"-0,5917","-0,7874","-0,9769",0,"-0,4554",...,"2,2581","-0,2275","2,7778","-0,9276","0,2146","-11,5385",0,"-0,6148","0,1444",0
03/01/2012,"1,5383","3,1009","3,4258","3,0206",0,"-1,6807","1,2987","1,5983",0,"0,4575",...,"2,2581","-0,2275","3,4304","-0,9276","0,2146","-11,5385",0,"-0,6148","0,1444",0
04/01/2012,"0,5374","2,3725","-0,849","0,4313",0,"-1,3177","1,1396","1,4566",0,"-0,6072",...,"0,4732","0,5701","-1,3065",0,"-0,2141","-11,5385",0,"3,6426","0,0481",0
05/01/2012,"1,1102","1,0219","0,0563","-1,3871",0,"-2,1292","3,5915","0,67",0,"-0,1222",...,"-2,1978","1,1338","1,6293","1,1236","0,4292","-11,5385",0,"-1,2599","0,2883",0
06/01/2012,"1,0454","1,5354","2,8152","-1,3642",0,"-0,7743","-1,1557","0,3613",0,"-0,8716",...,"0,9631","-2,4664","-4,7094","0,3704","-1,9231","-11,5385",0,"-1,2089","1,677",0


In [123]:
df_equity_m=df_equity_m.apply(lambda x: x.str.replace(',','.'))

In [124]:
df_equity_m=df_equity_m.astype(float)

In [125]:
df_equity_m.index = pd.to_datetime(df_equity_m.index)

In [128]:
print(df_equity_m)

            AAPL US Equity  MSFT US Equity  AMZN US Equity  GOOGL US Equity  \
Dates                                                                         
2012-02-01         -0.0296         -0.2306         -0.4371           0.5448   
2012-03-01          1.5383          3.1009          3.4258           3.0206   
2012-04-01          0.5374          2.3725         -0.8490           0.4313   
2012-05-01          1.1102          1.0219          0.0563          -1.3871   
2012-06-01          1.0454          1.5354          2.8152          -1.3642   
...                    ...             ...             ...              ...   
2022-09-03          3.4997          4.5858          2.4001           4.9687   
2022-10-03         -2.7186         -1.0087          5.4125          -0.7424   
2022-11-03         -2.3909         -1.9328         -0.8807          -1.9323   
2022-03-14         -2.6562         -1.2961         -2.5229          -3.0180   
2022-03-15         -2.6562         -1.2961         -

In [129]:
df_equity_m = df_equity_m.resample(rule='A').mean()

In [130]:
y=df_equity_m.stack()
y_df=pd.DataFrame(y)
y_df=y_df.reset_index(inplace=False)
y_df=y_df.rename(columns={"level_1": "TICKER"})

In [131]:
y_df.rename(columns = {0:'Return'}, inplace = True)

In [221]:
y_df.head(5)

,Dates,TICKER,Return
0,2012-12-31,AAPL US Equity,0.125975
1,2012-12-31,MSFT US Equity,0.051546
2,2012-12-31,AMZN US Equity,0.232471
3,2012-12-31,GOOGL US Equity,0.031781
4,2012-12-31,TSLA US Equity,0.037801


In [ ]:
def substract(a, b):                              
    return "".join(a.rsplit(b))

clean_columns = []
for i in df_equity.columns:
    clean_columns.append(substract(i, 'Equity'))
    
df_equity.set_axis(clean_columns, axis=1, inplace = True)
df_equity.head(5)

## Merge dataset
- Merge the data frame 
    - Return of the stocks and correspondance table data : key = TICKER
    - ESG historical data and correspondance table : key = ISIN
    - correspondance table (key = ISSUER) and financial data (key = Titre)
    - Merge all the 3 dataframe

### MERGE 1 - RETURN and CORRESPONDANCE

In [155]:
df_corespondance.rename(columns = {'TICKER EQUITY' : 'TICKER'}, inplace = True)
RETURN_CORRESP=pd.merge(df_corespondance, y_df, on='TICKER', how='inner')

In [156]:
RETURN_CORRESP.shape[0]

7756

In [157]:
RETURN_CORRESP.head(5)

,ISSUER,ISIN,TICKER,CDS 5 ANS,Dates,Return
0,Apple Inc,US0378331005,AAPL US Equity,CY179834,2012-12-31,0.125975
1,Apple Inc,US0378331005,AAPL US Equity,CY179834,2013-12-31,0.055808
2,Apple Inc,US0378331005,AAPL US Equity,CY179834,2014-12-31,0.149098
3,Apple Inc,US0378331005,AAPL US Equity,CY179834,2015-12-31,-0.008916
4,Apple Inc,US0378331005,AAPL US Equity,CY179834,2016-12-31,0.041784


In [165]:
len(list(RETURN_CORRESP['TICKER'].unique()))

738

### Merge 2 : RETURN_CORRESP and ESG HIST DATA

In [198]:
ESG_sub_criteria = ['Produced date','ISIN', ']
ESG_SCORE = df_hist_ESG[ESG_sub_criteria]

In [199]:
DATA_ESG_SCORE_RETURN =pd.merge(ESG_SCORE, RETURN_CORRESP, on='ISIN', how='inner')

In [200]:
DATA_ESG_SCORE_RETURN

,Produced date,ISIN,HRS Domain score,ENV score,C&S score,CIN score,CG score,HRts score,ISSUER,TICKER,CDS 5 ANS,Dates,Return
0,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2012-12-31,0.148033
1,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2013-12-31,0.127265
2,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2014-12-31,-0.092007
3,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2015-12-31,-0.132503
4,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2016-12-31,0.178651
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97977,01/03/2016,CH0011075394,29,48,42,30,51,50,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,2018-12-31,-0.111789
97978,01/03/2016,CH0011075394,29,48,42,30,51,50,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,2019-12-31,0.336643
97979,01/03/2016,CH0011075394,29,48,42,30,51,50,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,2020-12-31,0.260276
97980,01/03/2016,CH0011075394,29,48,42,30,51,50,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,2021-12-31,0.230128


### Merge 3 : correspondance table (key = ISSUER) and financial data (key = Titre)

In [225]:
usfull_cols = ['Titre', 'VM actu', 'Pays/Région', 'Secteur']
FINANCIAL_CORRESP = pd.merge(df_corespondance[['ISIN', 'ISSUER']], df_financial_data[usfull_cols], 
                             left_on='ISSUER', right_on='Titre')
FINANCIAL_CORRESP.shape

(596, 6)

In [226]:
FINANCIAL_CORRESP.head(5)

,ISIN,ISSUER,Titre,VM actu,Pays/Région,Secteur
0,US0378331005,Apple Inc,Apple Inc,181.50MLN,Etats-Unis,Technologie
1,US5949181045,Microsoft Corp,Microsoft Corp,142.16MLN,Etats-Unis,Technologie
2,US0231351067,Amazon.com Inc,Amazon.com Inc,93.53MLN,Etats-Unis,Conso discrétionnaire
3,US88160R1014,Tesla Inc,Tesla Inc,49.94MLN,Etats-Unis,Conso discrétionnaire
4,US91324P1021,UnitedHealth Group Inc,UnitedHealth Group Inc,32.30MLN,Etats-Unis,Santé


### Merge4 : DataFrame with all the parameters

In [235]:
DATA_FINAL = pd.merge(FINANCIAL_CORRESP, DATA_ESG_SCORE_RETURN, on='ISIN', how='right')

In [236]:
DATA_FINAL.shape[0]

97982

In [238]:
DATA_FINAL.head(2)

,ISIN,ISSUER_x,Titre,VM actu,Pays/Région,Secteur,Produced date,HRS Domain score,ENV score,C&S score,CIN score,CG score,HRts score,ISSUER_y,TICKER,CDS 5 ANS,Dates,Return
0,GB00B1YW4409,3i Group PLC,3i Group PLC,1.10MLN,Royaume-Uni,Financières,01/03/2007,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2012-12-31,0.148033
1,GB00B1YW4409,3i Group PLC,3i Group PLC,1.10MLN,Royaume-Uni,Financières,01/03/2007,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,2013-12-31,0.127265


# Machine Learning to predict RETURN using ESG CRITERIA

In [201]:
from sklearn.model_selection import train_test_split

In [202]:
def split_df(df, target = 'Return'):
    """
    Inputs : Dataframe and the name of the target column set by default to Return
    Output : dictionary of 4 DataFrames with keys : x_train,y_train,  x_test, y_test
    
    """
    
    # Create features and target
    x = df.drop(target, axis = 1)
    y = df[[target]].values

    # Echantillon de test (20%) et train (80%)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2 , random_state = 100, shuffle=False)
    
    return {'x_train' : x_train, 'x_test' : x_test, 'y_train' : y_train, 'y_test' : y_test}

In [203]:
DATA_ESG_SCORE_RETURN.columns

Index(['Produced date', 'ISIN', 'HRS Domain score', 'ENV score', 'C&S score',
       'CIN score', 'CG score', 'HRts score', 'ISSUER', 'TICKER', 'CDS 5 ANS',
       'Dates', 'Return'],
      dtype='object')

#### Model with ESG criteria only 

In [207]:
Input_columns = ['HRS Domain score','ENV score','C&S score', 'CIN score', 'CG score', 'HRts score', 'Return']
DATA = DATA_ESG_SCORE_RETURN[Input_columns]

In [208]:
# Training set
x_train = split_df(DATA)['x_train']
y_train = split_df(DATA)['y_train']

In [211]:
# Test set
x_test = split_df(DATA)['x_test']
y_test = split_df(DATA)['y_test']

In [240]:
reg = LinearRegression().fit(x_train, y_train)

In [216]:
#reg.coef_
reg.intercept_
coef = reg.coef_

In [217]:
coef

array([[-1.30183162e-04, -1.22327885e-04,  1.79937501e-04,
         1.89865925e-04,  4.93553742e-05, -1.34209867e-04]])

In [218]:
y_pred = reg.predict(x_test)

In [219]:
mean_squared_error(y_test, y_pred)

0.03683349050240753

In [245]:
model = sm.OLS(y_train,x_train)
results = model.fit()
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.067
Model:                            OLS   Adj. R-squared (uncentered):              0.067
Method:                 Least Squares   F-statistic:                              944.7
Date:                Fri, 22 Apr 2022   Prob (F-statistic):                        0.00
Time:                        22:54:44   Log-Likelihood:                          10472.
No. Observations:               78385   AIC:                                 -2.093e+04
Df Residuals:                   78379   BIC:                                 -2.088e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

### Model with ESG criteria and characteristics of the company

In [241]:
DATA_FINAL.columns

Index(['ISIN', 'ISSUER_x', 'Titre', 'VM actu', 'Pays/Région', 'Secteur',
       'Produced date', 'HRS Domain score', 'ENV score', 'C&S score',
       'CIN score', 'CG score', 'HRts score', 'ISSUER_y', 'TICKER',
       'CDS 5 ANS', 'Dates', 'Return'],
      dtype='object')

In [ ]:
Input_columns = ['HRS Domain score','ENV score','C&S score', 'CIN score', 'CG score', 'HRts score','VM actu',
                 'Pays/Région', 'Secteur', 'Return']

# FIN

## Merge dataset
- Merge the data frame 
    - ESG historical data and correspondance table : key = ISIN
    - coreespondance table (key = ISSUER) and financial data (key = Titre)
    - Merge all the 3 dataframe

In [102]:
global_score = ['ISIN', 'Global score']
global_score = ['ISIN', 'Global score']
ESG_criteria = ['Produced date','ISIN','ESG ENV score', 'ESG SOC score', 'ESG GOV score'] 
ESG_sub_criteria = ['Produced date','ISIN', 'HRS Domain score','ENV score','C&S score', 'CIN score', 'CG score', 'HRts score']
ESG_sub_sub_criteria = ['Produced date','ISIN', 
                        'HR L-score', 'HR I-score', 'HR R-score', 
                        'ENV L-score', 'ENV I-score','ENV R-score', 
                        'C&S L-score', 'C&S I-score', 'C&S R-score', 
                        'CIN score', 'CIN L-score', 'CIN I-score', 'CIN R-score',
                        'CG L-score', 'CG I-score', 'CG R-score', 
                        'HRts R-score', 'HRts I-score', 'HRts R-score']



In [103]:
df_hist_ESG[ESG_sub_criteria].head(3)

,Produced date,ISIN,HRS Domain score,ENV score,C&S score,CIN score,CG score,HRts score
0,01/09/2021,DE0005545503,32,30,34,15,38,47
1,01/11/2020,DE0005545503,13,17,29,9,29,42
2,01/09/2019,DE0005545503,13,20,29,9,25,45


In [59]:
df_hist_ESG[ESG_criteria].head(3)

,Produced date,ISIN,ESG ENV score,ESG SOC score,ESG GOV score
0,01/09/2021,DE0005545503,30.0,34.0,37.0
1,01/11/2020,DE0005545503,19.0,24.0,28.0
2,01/09/2019,DE0005545503,22.0,25.0,26.0


### Merge ESG hist data and Correspondance table

In [104]:
#Merging
ESG_CORRESP = pd.merge(df_hist_ESG[ESG_sub_criteria], df_corespondance, how='inner', on='ISIN')
ESG_CORRESP.shape

(9522, 12)

In [105]:
ESG_CORRESP = ESG_CORRESP.rename(columns={"Produced date": "Dates"})

In [62]:
ESG_CORRESP

,Dates,ISIN,HRS Domain score,ENV score,C&S score,CIN score,CG score,HRts score,ISSUER,TICKER EQUITY,CDS 5 ANS,TICKER
0,01/03/2007,GB00B1YW4409,20,47,45,50,68,21,3i Group PLC,FMC US Equity,CT355208,FMC US
1,01/03/2005,GB00B1YW4409,44,36,40,63,65,35,3i Group PLC,FMC US Equity,CT355208,FMC US
2,01/01/2020,GB00B1YW4409,40,50,54,42,71,59,3i Group PLC,FMC US Equity,CT355208,FMC US
3,01/03/2018,GB00B1YW4409,36,46,59,40,66,58,3i Group PLC,FMC US Equity,CT355208,FMC US
4,01/03/2016,GB00B1YW4409,35,44,49,24,62,52,3i Group PLC,FMC US Equity,CT355208,FMC US
...,...,...,...,...,...,...,...,...,...,...,...,...
9517,01/01/2020,CH0011075394,34,55,39,28,57,53,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,LRCX US
9518,01/07/2019,CH0011075394,34,55,40,28,57,53,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,LRCX US
9519,01/03/2018,CH0011075394,34,51,40,28,57,53,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,LRCX US
9520,03/07/2021,CH0011075394,37,59,34,31,69,54,Zurich Insurance Group AG,LRCX US Equity,CZURN1E5,LRCX US


In [63]:
df_corespondance.shape

(782, 5)

In [64]:
ESG_CORRESP.groupby('TICKER')['Dates'].count()

TICKER
11 HK       17
1299 HK     29
1801 JP      5
1812 JP     12
1925 JP      9
            ..
ZEN US       7
ZI US        7
ZS US        9
ZTS US      15
ZURN SW      8
Name: Dates, Length: 753, dtype: int64

In [ ]:
y_df

In [ ]:
ESG

In [133]:
df_corespondance = pd.read_excel('data/Correspondence Table.xlsx')
df_corespondance.shape

(4208, 24)

In [134]:
df_corespondance

,ISSUER,ISIN,TICKER,CDS 5 ANS,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,TICKER EQUITY,...,Unnamed: 14,TICKER BOND,ID ISIN.1,ISSUER.2,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,ISSUER EQUITY & BOND,ISIN EQUITY
0,Apple Inc,US0378331005,AAPL EU Equity,CY179834,NaN,NaN,NaN,NaN,NaN,AAPL US Equity,...,NaN,EJ346891 Corp,FR0011317783,FRANCE (GOVT OF),NaN,NaN,NaN,NaN,APPLE INC,US0378331005
1,Microsoft Corp,US5949181045,MSFTUSD EU Equity,CX399237,NaN,NaN,NaN,NaN,NaN,MSFT US Equity,...,NaN,EK243274 Corp,FR0011883966,FRANCE (GOVT OF),NaN,NaN,NaN,NaN,MICROSOFT CORP,US5949181045
2,Amazon.com Inc,US0231351067,AMZNUSD EU Equity,CY372412,NaN,NaN,NaN,NaN,NaN,AMZN US Equity,...,NaN,UV994928 Corp,FR0012993103,FRANCE (GOVT OF),NaN,NaN,NaN,NaN,JOHNSON & JOHNSON,US4781601046
3,Alphabet Inc,US02079K3059,GOOG1USD EU Equity,CX410247,NaN,NaN,NaN,NaN,NaN,GOOGL US Equity,...,NaN,EH878672 Corp,FR0010773192,FRANCE (GOVT OF),NaN,NaN,NaN,NaN,BERKSHIRE HATHAWAY INC,US0846707026
4,Tesla Inc,US88160R1014,TSLAUSD EU Equity,CY371958,NaN,NaN,NaN,NaN,NaN,GOOG US Equity,...,NaN,GG738427 Corp,FR0000571218,FRANCE (GOVT OF),NaN,NaN,NaN,NaN,JPMORGAN CHASE & CO,US46625H1005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,XS2247623643,GETLINK SE,NaN,NaN,NaN,NaN,NaN,NaN
4204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,XS2332306344,REXEL SA,NaN,NaN,NaN,NaN,NaN,NaN
4205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,XS2351382473,DERICHEBOURG,NaN,NaN,NaN,NaN,NaN,NaN
4206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,XS2399981435,SECHE ENVIRONNEMENT SA,NaN,NaN,NaN,NaN,NaN,NaN


### Merge financial data and correspondance table

In [106]:
usfull_cols = ['Titre', 'VM actu', 'Pays/Région', 'Secteur']

In [107]:
usfull_cols = ['Titre', 'VM actu', 'Pays/Région', 'Secteur']
FINANCIAL_CORRESP = pd.merge(df_corespondance, df_financial_data, left_on='ISSUER', right_on='Titre')
FINANCIAL_CORRESP.shape

(596, 21)

In [67]:
FINANCIAL_CORRESP.head(3)

,ISSUER,ISIN,TICKER EQUITY,CDS 5 ANS,TICKER,Titre,Ticker,Source,Position,Var pos,...,% Net,VM actu,Rpt MV,Var VM ac,Déposé le,Pays/Région,Secteur,Sous-secteur,Région,Type d'actif
0,Apple Inc,US0378331005,AAPL US Equity,CY179834,AAPL US,Apple Inc,AAPL US,ETF,"1,259,255",28,...,4.53,181.50MLN,172.74MLN,"+4,035.79",03/15/22,Etats-Unis,Technologie,Matériel tech et semi-conducteurs,Am. du Nord,Actions
1,Microsoft Corp,US5949181045,MSFT US Equity,CX399237,MSFT US,Microsoft Corp,MSFT US,ETF,547.454,13,...,3.62,142.16MLN,137.83MLN,"+3,375.77",03/15/22,Etats-Unis,Technologie,Logiciels et srvces tech,Am. du Nord,Actions
2,Amazon.com Inc,US0231351067,AMZN US Equity,CY372412,AMZN US,Amazon.com Inc,AMZN US,ETF,35.033,1,...,2.38,93.53MLN,90.52MLN,"+2,669.89",03/15/22,Etats-Unis,Conso discrétionnaire,Détail et gros - Conso discrét,Am. du Nord,Actions


### Merge ESG CORRESP and FINANCIAL CORRESP

In [108]:
ESG_FINANCIAL_CORRESP = pd.merge(ESG_CORRESP, FINANCIAL_CORRESP, how='left', on='ISIN')
ESG_FINANCIAL_CORRESP.shape

(9522, 32)

In [211]:
ESG_FINANCIAL_CORRESP.columns

Index(['Dates', 'ISIN', 'HRS Domain score', 'ENV score', 'C&S score',
       'CIN score', 'CG score', 'HRts score', 'ISSUER_x', 'CDS 5 ANS_x',
       'TICKER_x', 'ISSUER_y', 'CDS 5 ANS_y', 'TICKER_y', 'Titre', 'Ticker',
       'Source', 'Position', 'Var pos', '% circ', '% Net', 'VM actu', 'Rpt MV',
       'Var VM ac', 'Déposé le', 'Pays/Région', 'Secteur', 'Sous-secteur',
       'Région', 'Type d'actif'],
      dtype='object')

###  Merge ESG_FINANCIAL_CORRESP  and data with RETURN

In [110]:
ESG_FINANCIAL_CORRESP.rename(columns = {'TICKER EQUITY_x':'TICKER'}, inplace = True)

In [109]:
ESG_FINANCIAL_CORRESP.columns

Index(['Dates', 'ISIN', 'HRS Domain score', 'ENV score', 'C&S score',
       'CIN score', 'CG score', 'HRts score', 'ISSUER_x', 'TICKER EQUITY_x',
       'CDS 5 ANS_x', 'TICKER_x', 'ISSUER_y', 'TICKER EQUITY_y', 'CDS 5 ANS_y',
       'TICKER_y', 'Titre', 'Ticker', 'Source', 'Position', 'Var pos',
       '% circ', '% Net', 'VM actu', 'Rpt MV', 'Var VM ac', 'Déposé le',
       'Pays/Région', 'Secteur', 'Sous-secteur', 'Région', 'Type d'actif'],
      dtype='object')

In [111]:
ESG_FINANCIAL_CORRESP['Dates'] = pd.to_datetime(ESG_FINANCIAL_CORRESP['Dates'])

In [112]:
FINAL_DATA = pd.merge(ESG_FINANCIAL_CORRESP, y_df, on ='TICKER', how ='inner')

In [113]:
FINAL_DATA

,Dates_x,ISIN,HRS Domain score,ENV score,C&S score,CIN score,CG score,HRts score,ISSUER_x,TICKER,...,Rpt MV,Var VM ac,Déposé le,Pays/Région,Secteur,Sous-secteur,Région,Type d'actif,Dates_y,Return


In [114]:
df_hist_ESG

,Vigeo Key,ISIN,Bloomberg Legal entity identifier (LEI),Bloomberg Ticker,SEDOL code primary,SEDOL 2 code,Title,Zone,Country,Generic Sector,...,C&S score sector average,CIN score sector average,CG score sector average,HRts score sector average,HRS Domain rating,ENV Domain rating,C&S Domain rating,CIN Domain rating,CGV Domain rating,HRT Domain rating
0,DE0005545503,DE0005545503,5299003VKVDCUPSS5X23,1U1,5734672,NaN,1&1 AG,EUR,Germany,Telecommunications,...,38,33,50,51,=,-,=,-,-,=
1,DE0005545503,DE0005545503,5299003VKVDCUPSS5X23,1U1,5734672,NaN,1&1 Drillisch,EUR,Germany,Telecommunications,...,36,31,49,50,-,-,-,-,-,-
2,DE0005545503,DE0005545503,5299003VKVDCUPSS5X23,1U1,5734672,NaN,1&1 Drillisch,EUR,Germany,Telecommunications,...,36,32,47,49,--,-,-,--,-,=
3,DE0005545503,DE0005545503,5299003VKVDCUPSS5X23,1U1,5734672,NaN,1&1 Drillisch,EUR,Germany,Telecommunications,...,15,9,28,21,-,-,-,--,-,-
4,KVE00000509,US88025U1097,5493003Z6741WNLG7H43,TXG,BKS3RS7,NaN,"10X Genomics, Inc.",NAM,United States of America,Health Care Equipment & Services,...,33,29,40,33,-,-,-,-,=,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34667,XS0520578096,XS0520578096,529900UUANUJ60MNMX97,NaN,NaN,NaN,ÖBB-Infrastruktur AG,EUR,Austria,Transport & Logistics,...,32,31,41,37,-,-,--,-,--,-
34668,XS0520578096,XS0520578096,529900UUANUJ60MNMX97,NaN,NaN,NaN,ÖBB-Infrastruktur AG,EUR,Austria,Transport & Logistics,...,34,30,45,39,-,-,-,-,--,-
34669,XS0520578096,XS0520578096,529900UUANUJ60MNMX97,NaN,NaN,NaN,ÖBB-Infrastruktur AG,EUR,Austria,Transport & Logistics,...,32,31,41,37,-,-,-,-,--,-
34670,AT0000APOST4,AT0000APOST4,529900MVUWACNUTK8467,POST,B1577G7,NaN,Österreichische Post AG,EUR,Austria,Transport & Logistics,...,34,30,45,39,=,=,-,=,+,=


Regression

In [ ]:
x

In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(Y,X)
results = model.fit()

# Step 1: critical analysis of data

- Statistical quality (homogeneity of methods,particularly management methods, volume, representativeness, traceability)
- Possible consequences of data quality on the results of the studies
- Very different situation: little data, from several information systems, with missing data, requiring a lot or reprocessing before being exploited. Sometimes no "relevant" data will be available
- Checked that the data used meet the following 3 criteria: Relevance, Completeness, Accuracy
- ensure that the data used does not have a significant impact on the calculations or the results obtained

# Step 2: ESG rating criteria classification
link between sustainability and trend in world prices & CDS
- Define which ESG sub-criteria have the strongest financial impact on a company and vice versa
- Can ESG data be used to forecast prices or CDS